In [108]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import os.path
import pandas as pd
import numpy as np

#### Selecionando a lista de ações através de um dataset de ações da B3

In [109]:
df_acoes = pd.read_csv('arquivos/acoes-listadas-b3.csv')
todas_as_acoes = df_acoes['Ticker'].to_list()
todas_as_acoes_nparray = np.array(todas_as_acoes)
#selecione as 300 primeiras ações
lista_acoes2 = todas_as_acoes_nparray[0:300]


In [110]:
lista_acoes = np.array([]) # Aqui eu colei todos os nomes das ações e, a medida que erros iam acontecendo, eu ia removendo os nomes das ações que já tinham sido inseridas no csv

len(lista_acoes)

0

#### Código do bot que visitava o site investidor10 para coletar os resultados trimestrais das empresas

In [111]:
def bot():
    servico = Service(ChromeDriverManager().install())
    lista_acoes_validas = []
    acoes_com_erro = []
    navegador = webdriver.Chrome(service=servico)
    navegador.maximize_window()
    for acao in lista_acoes:


        navegador.get(f"https://investidor10.com.br/acoes/{acao}/")

        tabela_resultados = navegador.find_element(By.ID, "results_table")
        # desse a tela ate o elemento
        navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)
        time.sleep(2)
        # verifica se ta visivil
        while not tabela_resultados.is_displayed():
            navegador.execute_script("arguments[0].scrollIntoView();", tabela_resultados)

        time.sleep(2)

        botoes = tabela_resultados.find_element(By.CLASS_NAME, "nav")
        botoes = botoes.find_elements(By.TAG_NAME, "li")


        bt_trimestral = botoes[3]
        bt_trimestral.click()
        time.sleep(1)

        dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
        dropdow = dropdow.find_elements(By.TAG_NAME, "li")
        dropdow[1].click()
        time.sleep(1)

        bt_valores = botoes[1]
        bt_valores.click()
        time.sleep(0.5)
        # clica no botao onde tem o texto "Valores Detalhados"
        dropdow = navegador.find_element(By.CLASS_NAME, "select2-results__options")
        dropdow = dropdow.find_elements(By.TAG_NAME, "li")
        dropdow[1].click()

        time.sleep(1)

        tabela = navegador.find_element(By.ID, "table-balance-results")

        linhas = tabela.find_elements(By.TAG_NAME, "tr")



        # A partir daqui, os dados estao sendo coletados
        # salve esses dados em um arquivo CSV
        colunas = linhas[0].find_elements(By.TAG_NAME, "th")
        if not os.path.isfile('arquivos/dataset_ML.csv'): # Para nao escrever as colunas novamente
            with open('arquivos/dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
                f.write("Empresa;Resultado;")
                for coluna in colunas:
                    if coluna.text == "" or coluna.text == "#":
                        continue
                    # inserindo uma condiçao para nao colocar o ";" na ultima coluna
                    if coluna.text == "1T2019":
                        linha = coluna.text + "\n"
                    else:
                        linha = coluna.text + ";"
                    f.write(linha)
                    print(coluna.text, end=" ")
                print()
                f.close()
        with open('arquivos/dataset_ML.csv', 'a', newline='', encoding='utf-8') as f:
            indices_desejados = [1,4,6,9,13]
            for idx, linha in enumerate(linhas[1:], start=1):
                if idx in indices_desejados:
                    colunas = linha.find_elements(By.TAG_NAME, "td")
                    i = 1
                    if len(colunas) !=60: #Significa que estão faltando trimestres nos dados da ação
                        acoes_com_erro.append(acao)
                        break #Passo para a próxima ação
                    f.write(acao + ";")
                    if acao not in lista_acoes_validas:
                        lista_acoes_validas.append(acao)
                    for coluna in colunas:
                        
                        if coluna.text == "":
                            continue
                        #faça uma condicao que verifique coluna.text é o ultimo elemento e nao coloque o ";"
                        
                        if i == 21:
                            linha = coluna.text + "\n"
                        else:
                            linha = coluna.text + ";"
                        i += 1
                        f.write(linha)

                        print(coluna.text, end=" ")
            
                    print()

    lista_acoes = lista_acoes_validas
    return lista_acoes, acoes_com_erro

### Funções usadas para criar o dataset final, que contém os resultados trimestrais de 2023 e uma classificação: 
- 0 = ação desvalorizou.
- 1 = ação valorizou menos de 10%.
- 2 = ação valorizou mais de 10%.

In [112]:
def criacao_dataframe_3trimestres(df, index_col, columns_col, reindex_list):
    pivot_table = df.pivot_table(index=index_col, columns=columns_col, aggfunc='sum')
    pivot_table = pivot_table.reindex(reindex_list)
    pivot_table.columns = ['_'.join(col).strip() for col in pivot_table.columns.values] # Transformando o multi index da coluna em um index simples
    pivot_table = pivot_table.filter(like='23', axis=1) # Filtrando apenas os resultados do ano de 2023
    pivot_table = pivot_table.drop(pivot_table.filter(like='4T').columns, axis=1) # Removendo os resultados do 4T de 2023

    return pivot_table

"""Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para 
reindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre (menos o 4T), filtrando pelo ano de 2023, que servirá como base de dados para análise"""

'Essa função cria uma tabela pivot a partir de um dataframe, onde index_col é a coluna que será utilizada como índice (Empresa), columns_col é a coluna que será utilizada como coluna (Resultado) e reindex_list é a lista que será utilizada para \nreindexar a tabela pivot (lista_acoes),pois a ordem original foi alterada. Eu estou colocando como índice cada empresa e como coluna os resultados de cada trimestre (menos o 4T), filtrando pelo ano de 2023, que servirá como base de dados para análise'

In [113]:
def criacao_dataframe_dados_historicos(diretorio, lista_acoes, mes_de_valorizacao):
    dataframes = []
    for acao in lista_acoes:
        df = pd.read_csv(f'{diretorio}/{acao} Dados Históricos.csv')
        #Inserindo o nome da empresa na coluna "Empresa" e colocando ela como index
        df['Empresa'] = acao
        df.set_index('Empresa', inplace=True)
        # Selecionando apenas o mes de novembro de 2023
        df = df[df['Data'] == mes_de_valorizacao]
        # Calculando a variação - convertendo os valores em float
        df['Último'] = df['Último'].str.replace(',', '.').astype(float)
        df['Abertura'] = df['Abertura'].str.replace(',', '.').astype(float)
        df['Variação'] = (df['Último'] - df['Abertura']) / df['Abertura'] * 100
        # Criando a classificação, que vai servir como o Y do modelo
        if (df['Variação'] > 0).any():
            if (df['Variação'] > 10).any():
                df['Classificação'] = 2
            else:
                df['Classificação'] = 1
        else:
            df['Classificação'] = 0
        dataframes.append(df)
    df_final = pd.concat(dataframes)
    return df_final

"""Essa função está selecionando os arquivos csv dentro da pasta 'dados historicos' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, 
selecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, 
e se subiu mais de 10%, a classificação é 2."""

'Essa função está selecionando os arquivos csv dentro da pasta \'dados historicos\' de acorco com a lista_acoes. Para cada arquivo csv, ela está inserindo o nome da empresa na coluna "Empresa" e colocando ela no começo do dataframe, \nselecionando apenas o mês de novembro de 2023, calculando a variação da ação nesse mês e criando a classificação, que vai servir como o Y do modelo. Se a empresa caiu, a classificação é 0, se subiu até 10%, a classificação é 1, \ne se subiu mais de 10%, a classificação é 2.'

In [114]:
def atribuir_classificacao(dataframe_resultados, dataframe_dados_historicos):
    dataframe_final = dataframe_resultados
    dataframe_final['Classificação'] = dataframe_dados_historicos['Classificação']
    return dataframe_final

# Essa função está atribuindo a coluna classificação do dataframe de dados históricos ao dataframe de resultados

Sobre as colunas do dataset:
- **Receita Líquida:** é a quantia de dinheiro que uma empresa ganha com suas vendas após a dedução de devoluções, descontos e impostos sobre vendas.
- **Lucro Líquido:** representa o lucro de uma empresa após todas as despesas e impostos terem sido deduzidos da receita total.
- **EBIT:** representa a lucratividade de uma empresa antes da consideração dos juros e impostos.
- **Dívida Líquida:** a diferença entre todos os compromissos financeiros pendentes e os recursos financeiros imediatamente disponíveis para liquidá-los.
- **ROE:** é uma medida que revela a capacidade de uma empresa gerar lucro a partir do dinheiro que os acionistas investiram nela.


In [115]:
# Ler o arquivo CSV
df = pd.read_csv('arquivos/dataset_ML.csv', delimiter=';')
df2 = df.copy()

## Criando o dataframe em que cada coluna representa um resultado de um trimestre de 2023

In [116]:
#Obtendo os valores unicos da coluna 'Empresa' para atribuir para lista_acoes
lista_acoes = df2['Empresa'].unique()
dataframe_3trimestres = criacao_dataframe_3trimestres(df2, 'Empresa', 'Resultado', lista_acoes)
# Removendo as linhas que possuem alguma celula com um '-'
dataframe_3trimestres = dataframe_3trimestres.replace('-', np.nan)
dataframe_3trimestres = dataframe_3trimestres.dropna()
acoes_do_dataframe = np.array(dataframe_3trimestres.index) #Obtendo as ações do dataframe após a limpeza das que possuíam dados faltantes
dataframe_3trimestres


,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),3T2023_Dívida Líquida - (R$),3T2023_EBIT - (R$),3T2023_Lucro Líquido - (R$),3T2023_ROE - (%),3T2023_Receita Líquida - (R$)
Empresa,,,,,,,,,,,,,,,
PETR4,204.125.000.000,60.197.000.000,38.307.000.000,"9,49 %",139.068.000.000,218.390.000.000,41.971.000.000,28.936.000.000,"7,76 %",113.840.000.000,238.304.000.000,47.546.000.000,26.760.000.000,"6,90 %",124.828.000.000
MGLU3,5.038.335.000,16.328.000,-391.221.000,"-3,80 %",9.067.334.000,4.744.924.000,-35.923.000,-301.746.000,"-3,01 %",8.572.256.000,4.118.464.000,661.189.000,-498.330.000,"-5,23 %",8.578.818.000
OIBR3,20.956.798.000,-102.677.000,-1.267.087.000,"-5,47 %",2.535.792.000,21.232.781.000,-266.082.000,-844.524.000,"-3,53 %",2.454.171.000,22.719.925.000,-803.758.000,-2.830.492.000,"-10,55 %",2.421.776.000
B3SA3,-2.501.944.000,1.356.975.000,1.089.458.000,"5,30 %",2.460.480.000,-2.834.396.000,1.374.315.000,1.052.942.000,"5,13 %",2.477.122.000,-3.353.958.000,1.347.206.000,1.074.307.000,"5,31 %",2.490.082.000
COGN3,6.158.625.000,259.730.000,53.920.000,"0,45 %",1.329.876.000,6.161.440.000,147.387.000,-64.907.000,"-0,39 %",1.386.677.000,6.077.436.000,90.916.000,-116.656.000,"-0,85 %",1.270.040.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3,177.193.000,-7.610.000,-17.872.000,"-30,31 %",86.779.000,167.306.000,1.138.000,-14.127.000,"-22,00 %",100.934.000,178.597.000,-597.000,-15.005.000,"-45,80 %",94.225.000
NEXP3,48.337.000,-13.298.000,-17.843.000,"-18,42 %",30.066.000,50.187.000,26.464.000,29.871.000,"-23,20 %",37.420.000,50.369.000,4.650.000,4.411.000,"-4,09 %",-2.473.000
EMAE4,-447.375.000,1.038.000,18.882.000,"1,65 %",137.552.000,-468.834.000,10.281.000,37.848.000,"3,39 %",138.754.000,-457.016.000,39.526.000,41.102.000,"3,55 %",173.596.000


## Criando o dataframe que contém a variação do preço das ações em novembro, e que vai gerar coluna classificação 

In [117]:
dataframe_variacao_preco_acoes = criacao_dataframe_dados_historicos('dados historicos', acoes_do_dataframe, '01.11.2023')
dataframe_variacao_preco_acoes

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Variação,Classificação
Empresa,,,,,,,,,
PETR4,01.11.2023,35.91,33.73,"36,03","32,66","992,20M","3,37%",6.463089,1
MGLU3,01.11.2023,2.02,1.33,"2,35","1,31","4,53B","51,88%",51.879699,2
OIBR3,01.11.2023,0.65,0.60,"0,75","0,59","120,11M","10,17%",8.333333,1
B3SA3,01.11.2023,13.30,11.14,"13,45","11,13","729,75M","19,82%",19.389587,2
COGN3,01.11.2023,3.23,2.41,"3,26","2,39","528,64M","34,58%",34.024896,2
...,...,...,...,...,...,...,...,...,...
BAHI3,01.11.2023,8.06,9.15,"9,91","7,90","68,70K","-10,44%",-11.912568,0
NEXP3,01.11.2023,5.36,4.93,"5,70","4,91","24,60K","8,72%",8.722110,1
EMAE4,01.11.2023,59.51,59.39,"62,93","55,06","147,00K","0,02%",0.202054,1


## Criando o dataframe com os resultados trimestrais e a classificação

In [118]:
dataframe_dados_completos = atribuir_classificacao(dataframe_3trimestres, dataframe_variacao_preco_acoes)
dataframe_dados_completos

,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),3T2023_Dívida Líquida - (R$),3T2023_EBIT - (R$),3T2023_Lucro Líquido - (R$),3T2023_ROE - (%),3T2023_Receita Líquida - (R$),Classificação
Empresa,,,,,,,,,,,,,,,,
PETR4,204.125.000.000,60.197.000.000,38.307.000.000,"9,49 %",139.068.000.000,218.390.000.000,41.971.000.000,28.936.000.000,"7,76 %",113.840.000.000,238.304.000.000,47.546.000.000,26.760.000.000,"6,90 %",124.828.000.000,1
MGLU3,5.038.335.000,16.328.000,-391.221.000,"-3,80 %",9.067.334.000,4.744.924.000,-35.923.000,-301.746.000,"-3,01 %",8.572.256.000,4.118.464.000,661.189.000,-498.330.000,"-5,23 %",8.578.818.000,2
OIBR3,20.956.798.000,-102.677.000,-1.267.087.000,"-5,47 %",2.535.792.000,21.232.781.000,-266.082.000,-844.524.000,"-3,53 %",2.454.171.000,22.719.925.000,-803.758.000,-2.830.492.000,"-10,55 %",2.421.776.000,1
B3SA3,-2.501.944.000,1.356.975.000,1.089.458.000,"5,30 %",2.460.480.000,-2.834.396.000,1.374.315.000,1.052.942.000,"5,13 %",2.477.122.000,-3.353.958.000,1.347.206.000,1.074.307.000,"5,31 %",2.490.082.000,2
COGN3,6.158.625.000,259.730.000,53.920.000,"0,45 %",1.329.876.000,6.161.440.000,147.387.000,-64.907.000,"-0,39 %",1.386.677.000,6.077.436.000,90.916.000,-116.656.000,"-0,85 %",1.270.040.000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3,177.193.000,-7.610.000,-17.872.000,"-30,31 %",86.779.000,167.306.000,1.138.000,-14.127.000,"-22,00 %",100.934.000,178.597.000,-597.000,-15.005.000,"-45,80 %",94.225.000,0
NEXP3,48.337.000,-13.298.000,-17.843.000,"-18,42 %",30.066.000,50.187.000,26.464.000,29.871.000,"-23,20 %",37.420.000,50.369.000,4.650.000,4.411.000,"-4,09 %",-2.473.000,1
EMAE4,-447.375.000,1.038.000,18.882.000,"1,65 %",137.552.000,-468.834.000,10.281.000,37.848.000,"3,39 %",138.754.000,-457.016.000,39.526.000,41.102.000,"3,55 %",173.596.000,1


In [119]:
# Colunas em que os valores serão mudados de 1.000.000 para 1000000
colunas_para_retirar_o_ponto = np.array(dataframe_dados_completos.columns)
colunas_para_retirar_o_ponto = np.delete(colunas_para_retirar_o_ponto, [3, 8, 13, 15])

### Realizando o tratamento dos dados do dataset

In [120]:
# Substituindo as '%' por '' e as ',' por '.' e convertendo para float
dataframe_dados_completos = dataframe_dados_completos.replace('%', '', regex=True)
dataframe_dados_completos = dataframe_dados_completos.replace(',', '.', regex=True)
# Agora eu preciso tirar os '.' para conseguir converter o numero para float, por exemplo, 1.000.000 para 1000000, isso deve ser feito em 'colunas_para_retirar_o_ponto'
for coluna in colunas_para_retirar_o_ponto:
    dataframe_dados_completos[coluna] = dataframe_dados_completos[coluna].str.replace('.', '')

dataframe_dados_completos = dataframe_dados_completos.astype(float)
#Convertento a coluna 'Classificação' para int
dataframe_dados_completos['Classificação'] = dataframe_dados_completos['Classificação'].astype(int)
dataframe_dados_completos


,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),3T2023_Dívida Líquida - (R$),3T2023_EBIT - (R$),3T2023_Lucro Líquido - (R$),3T2023_ROE - (%),3T2023_Receita Líquida - (R$),Classificação
Empresa,,,,,,,,,,,,,,,,
PETR4,2.041250e+11,6.019700e+10,3.830700e+10,9.49,1.390680e+11,2.183900e+11,4.197100e+10,2.893600e+10,7.76,1.138400e+11,2.383040e+11,4.754600e+10,2.676000e+10,6.90,1.248280e+11,1
MGLU3,5.038335e+09,1.632800e+07,-3.912210e+08,-3.80,9.067334e+09,4.744924e+09,-3.592300e+07,-3.017460e+08,-3.01,8.572256e+09,4.118464e+09,6.611890e+08,-4.983300e+08,-5.23,8.578818e+09,2
OIBR3,2.095680e+10,-1.026770e+08,-1.267087e+09,-5.47,2.535792e+09,2.123278e+10,-2.660820e+08,-8.445240e+08,-3.53,2.454171e+09,2.271992e+10,-8.037580e+08,-2.830492e+09,-10.55,2.421776e+09,1
B3SA3,-2.501944e+09,1.356975e+09,1.089458e+09,5.30,2.460480e+09,-2.834396e+09,1.374315e+09,1.052942e+09,5.13,2.477122e+09,-3.353958e+09,1.347206e+09,1.074307e+09,5.31,2.490082e+09,2
COGN3,6.158625e+09,2.597300e+08,5.392000e+07,0.45,1.329876e+09,6.161440e+09,1.473870e+08,-6.490700e+07,-0.39,1.386677e+09,6.077436e+09,9.091600e+07,-1.166560e+08,-0.85,1.270040e+09,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BAHI3,1.771930e+08,-7.610000e+06,-1.787200e+07,-30.31,8.677900e+07,1.673060e+08,1.138000e+06,-1.412700e+07,-22.00,1.009340e+08,1.785970e+08,-5.970000e+05,-1.500500e+07,-45.80,9.422500e+07,0
NEXP3,4.833700e+07,-1.329800e+07,-1.784300e+07,-18.42,3.006600e+07,5.018700e+07,2.646400e+07,2.987100e+07,-23.20,3.742000e+07,5.036900e+07,4.650000e+06,4.411000e+06,-4.09,-2.473000e+06,1
EMAE4,-4.473750e+08,1.038000e+06,1.888200e+07,1.65,1.375520e+08,-4.688340e+08,1.028100e+07,3.784800e+07,3.39,1.387540e+08,-4.570160e+08,3.952600e+07,4.110200e+07,3.55,1.735960e+08,1


In [121]:
# Salvando o dataframe em um arquivo CSV na pasta instancias modelos
dataframe_dados_completos.to_csv('instancias modelos/dataset_3T2023.csv')

# Criando um dataset com dois trimestres para testes

In [122]:
# Excluindo as colunas que possuírem 3T do dataset dataframe_dados_completos e a coluna 'Classificação'
df_2t = dataframe_dados_completos.drop(dataframe_dados_completos.filter(like='3T').columns, axis=1)
df_2t = df_2t.drop('Classificação', axis=1)
df_2t

# Chamando a função criacao_dataframe_dados_historicos que vai ter a coluna classificação
dataframe_variacao_preco_acoes_2t = criacao_dataframe_dados_historicos('dados historicos', acoes_do_dataframe, '01.08.2023')

# Chamando a função atribuir_classificacao
dataframe_dados_completos_2t = atribuir_classificacao(df_2t, dataframe_variacao_preco_acoes_2t)
dataframe_dados_completos_2t

,1T2023_Dívida Líquida - (R$),1T2023_EBIT - (R$),1T2023_Lucro Líquido - (R$),1T2023_ROE - (%),1T2023_Receita Líquida - (R$),2T2023_Dívida Líquida - (R$),2T2023_EBIT - (R$),2T2023_Lucro Líquido - (R$),2T2023_ROE - (%),2T2023_Receita Líquida - (R$),Classificação
Empresa,,,,,,,,,,,
PETR4,2.041250e+11,6.019700e+10,3.830700e+10,9.49,1.390680e+11,2.183900e+11,4.197100e+10,2.893600e+10,7.76,1.138400e+11,1
MGLU3,5.038335e+09,1.632800e+07,-3.912210e+08,-3.80,9.067334e+09,4.744924e+09,-3.592300e+07,-3.017460e+08,-3.01,8.572256e+09,0
OIBR3,2.095680e+10,-1.026770e+08,-1.267087e+09,-5.47,2.535792e+09,2.123278e+10,-2.660820e+08,-8.445240e+08,-3.53,2.454171e+09,0
B3SA3,-2.501944e+09,1.356975e+09,1.089458e+09,5.30,2.460480e+09,-2.834396e+09,1.374315e+09,1.052942e+09,5.13,2.477122e+09,0
COGN3,6.158625e+09,2.597300e+08,5.392000e+07,0.45,1.329876e+09,6.161440e+09,1.473870e+08,-6.490700e+07,-0.39,1.386677e+09,0
...,...,...,...,...,...,...,...,...,...,...,...
BAHI3,1.771930e+08,-7.610000e+06,-1.787200e+07,-30.31,8.677900e+07,1.673060e+08,1.138000e+06,-1.412700e+07,-22.00,1.009340e+08,0
NEXP3,4.833700e+07,-1.329800e+07,-1.784300e+07,-18.42,3.006600e+07,5.018700e+07,2.646400e+07,2.987100e+07,-23.20,3.742000e+07,0
EMAE4,-4.473750e+08,1.038000e+06,1.888200e+07,1.65,1.375520e+08,-4.688340e+08,1.028100e+07,3.784800e+07,3.39,1.387540e+08,0


In [123]:
dataframe_dados_completos_2t.to_csv('instancias modelos/dataset_2T2023.csv')